In [9]:
# Basic libraries
import pandas as pd
import numpy as np
import os
import time
import json
import pickle

# PyTorch related
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Transformers library
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel, BertForSequenceClassification

# Date and time processing
from pandas.tseries.offsets import DateOffset
from dateutil.relativedelta import relativedelta

# Progress bar
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# Evaluation metrics
from sklearn.metrics import r2_score, mean_squared_error

# Date offset
from pandas.tseries.offsets import DateOffset

# import data

In [11]:
news_df = pd.read_csv(r"C:\Users\Admin\Documents\FA690_MiniProject2\news.csv")
news_df["publication_datetime"] = pd.to_datetime(news_df["publication_datetime"])

price_df = pd.read_csv(r"C:\Users\Admin\Documents\FA690_MiniProject2\price.csv")
price_df["daily_return"] = price_df.groupby("ticker")["close"].transform(
    lambda x: x.pct_change()
)
price_df = price_df.dropna()
price_df["Date"] = pd.to_datetime(price_df["Date"])

In [12]:
news_df

,publication_datetime,title,body,tickers
0,2017-01-03,World News: Police Question Netanyahu Over Gifts,"""We pay attention to publications in the media...",EL
1,2017-01-03,Business News: Nestle Turns to New CEO for Hea...,"Nestle, the world's largest packaged-food comp...",GIS
2,2017-01-03,Business News: Vermont Drug Law Faces Limits -...,"The Vermont law, enacted in June, instructed s...",ABBV
3,2017-01-03,Life & Arts -- Travel: How Hotel Companies Lau...,Travelers are about to see a flurry of new hot...,HLT
4,2017-01-03,Businesses Ready to Ramp Up Investment --- Aft...,The Federal Reserve last month signaled intere...,HD
...,...,...,...,...
20545,2020-12-30,Terms in Google Ad Deal Revealed,"Ten Republican attorneys general, led by Texas...",GOOGL
20546,2020-12-30,'Son of Sam' Law Invoked in Phony Heiress Case,"Last year, the New York state attorney general...",NFLX
20547,2020-12-30,Boeing MAX Returns to U.S. Sky,Daily round-trip flights between Miami and New...,BA
20548,2020-12-30,"To Curb Ma's Empire, China Weighs Taking a Big...","The regulators, led by the central bank, also ...",PYPL


# Data Sampling
Consider two sceneraios:
 - time-series split:  
        Train: up to 2019-12-31  
        Validation: 2020-01-01 to 2020-06-30  
        Test: from 2020-07-01 onward
    - The issue with this is that validation set has a different distribution with training set due to COVID
- rolling-window split:  
        Train: 12month  
        Validation: 4month  
        Test: 2month  
        Rolling step: 2month

In [13]:
# Define window sizes
train_months = 12
val_months = 4
test_months = 2
step_months = 2

# Find the earliest and latest datetime
start_date = pd.to_datetime("2017-01-01")
end_date = pd.to_datetime("2020-12-31")

# Initialize result list
rolling_splits = []

current_start = start_date

while True:
    train_start = current_start
    train_end = train_start + relativedelta(months=train_months) - pd.Timedelta(days=1)

    val_start = train_end + pd.Timedelta(days=1)
    val_end = val_start + relativedelta(months=val_months) - pd.Timedelta(days=1)

    test_start = val_end + pd.Timedelta(days=1)
    test_end = test_start + relativedelta(months=test_months) - pd.Timedelta(days=1)

    # Stop if test_end exceeds data range
    if test_end > end_date:
        break

    train_set = news_df[
        (news_df["publication_datetime"] >= train_start)
        & (news_df["publication_datetime"] <= train_end)
    ]
    val_set = news_df[
        (news_df["publication_datetime"] >= val_start)
        & (news_df["publication_datetime"] <= val_end)
    ]
    test_set = news_df[
        (news_df["publication_datetime"] >= test_start)
        & (news_df["publication_datetime"] <= test_end)
    ]

    years = {
        "train": pd.date_range(train_start, train_end, freq="MS")
        .strftime("%Y-%m")
        .tolist(),
        "val": pd.date_range(val_start, val_end, freq="MS").strftime("%Y-%m").tolist(),
        "test": pd.date_range(test_start, test_end, freq="MS")
        .strftime("%Y-%m")
        .tolist(),
    }

    rolling_splits.append(
        {
            "train_set": train_set,
            "val_set": val_set,
            "test_set": test_set,
            "years": years,
        }
    )

    # Move forward by rolling step
    current_start += relativedelta(months=step_months)

In [14]:
rolling_splits[-1]

{'train_set':       publication_datetime                                              title  \
 13057           2019-07-01  Streetwise: Fund's Ethical Impact Is Difficult...   
 13058           2019-07-01  Markets Review & Outlook: Second Quarter --- U...   
 13059           2019-07-01  Markets Review & Outlook: Second Quarter --- U...   
 13060           2019-07-01  Markets Review & Outlook: Second Quarter --- S...   
 13061           2019-07-01  Jony Ive's Long Drift From Apple --- The desig...   
 ...                    ...                                                ...   
 18034           2020-06-30  Business News: Cloud Arm Zeros In on Space Org...   
 18035           2020-06-30  Business News: Cloud Arm Zeros In on Space Org...   
 18036           2020-06-30             Casinos Are Sued Over Virus Protection   
 18037           2020-06-30  AMC Delays Reopening Theaters As Studios Push ...   
 18038           2020-06-30                                          Overheard   
 
 

In [15]:
# Print the number of splits
print(f"Number of splits: {len(rolling_splits)}")

# Summarize each split
for i, split in enumerate(rolling_splits): 
    train_months = len(split['years']['train'])
    val_months = len(split['years']['val'])
    test_months = len(split['years']['test'])
    
    train_rows = len(split['train_set'])
    val_rows = len(split['val_set'])
    test_rows = len(split['test_set'])
    
    print(f"\nSplit {i+1}:")
    print(f"  Train: {split['years']['train'][0]} to {split['years']['train'][-1]} ({train_months} months, {train_rows} articles)")
    print(f"  Val:   {split['years']['val'][0]} to {split['years']['val'][-1]} ({val_months} months, {val_rows} articles)")
    print(f"  Test:  {split['years']['test'][0]} to {split['years']['test'][-1]} ({test_months} months, {test_rows} articles)")

Number of splits: 16

Split 1:
  Train: 2017-01 to 2017-12 (12 months, 5015 articles)
  Val:   2018-01 to 2018-04 (4 months, 1839 articles)
  Test:  2018-05 to 2018-06 (2 months, 902 articles)

Split 2:
  Train: 2017-03 to 2018-02 (12 months, 5103 articles)
  Val:   2018-03 to 2018-06 (4 months, 1916 articles)
  Test:  2018-07 to 2018-08 (2 months, 848 articles)

Split 3:
  Train: 2017-05 to 2018-04 (12 months, 5308 articles)
  Val:   2018-05 to 2018-08 (4 months, 1750 articles)
  Test:  2018-09 to 2018-10 (2 months, 884 articles)

Split 4:
  Train: 2017-07 to 2018-06 (12 months, 5365 articles)
  Val:   2018-07 to 2018-10 (4 months, 1732 articles)
  Test:  2018-11 to 2018-12 (2 months, 832 articles)

Split 5:
  Train: 2017-09 to 2018-08 (12 months, 5350 articles)
  Val:   2018-09 to 2018-12 (4 months, 1716 articles)
  Test:  2019-01 to 2019-02 (2 months, 862 articles)

Split 6:
  Train: 2017-11 to 2018-10 (12 months, 5334 articles)
  Val:   2018-11 to 2019-02 (4 months, 1694 articles)


# Pretrained Large Language Model: FinBERT

In [16]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load FinBERT model and tokenizer: https://huggingface.co/yiyanghkust/finbert-tone
finbert_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--yiyanghkust--finbert-tone. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [17]:
# print the maximum length of input tokens
print(f"The maximum length of input tokens for FinBERT: {finbert.config.max_position_embeddings}")

The maximum length of input tokens for FinBERT: 512


In [18]:
def predict_sentiment(text):
    inputs = finbert_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = finbert(**inputs)
    scores = outputs.logits.softmax(dim=1).numpy()[0]
    return scores

In [19]:
# Convert the list of scores into a DataFrame with separate columns
sentiment_df = pd.DataFrame(news_df['body'].apply(predict_sentiment).tolist(), columns=['neutral', 'positive', 'negative'])

# Append the new columns to news_df
news_df = pd.concat([news_df.reset_index(drop=True), sentiment_df], axis=1)

# save FinBERT sentiment
news_df.to_csv("news_w_sentiment_FinBERT.csv", index = False)
news_df.tail()

,publication_datetime,title,body,tickers,neutral,positive,negative
20545,2020-12-30,Terms in Google Ad Deal Revealed,"Ten Republican attorneys general, led by Texas...",GOOGL,0.233786,0.031028,0.735186
20546,2020-12-30,'Son of Sam' Law Invoked in Phony Heiress Case,"Last year, the New York state attorney general...",NFLX,0.999567,0.000021,0.000413
20547,2020-12-30,Boeing MAX Returns to U.S. Sky,Daily round-trip flights between Miami and New...,BA,0.999890,0.000064,0.000045
20548,2020-12-30,"To Curb Ma's Empire, China Weighs Taking a Big...","The regulators, led by the central bank, also ...",PYPL,0.996614,0.002923,0.000463
20549,2020-12-30,Slack Deal Shows Handcuffs on Tech Firms,Salesforce.com's $27.7 billion acquisition of ...,CRM,0.973938,0.024925,0.001138


# Fine-tune BERT

In [20]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

In [21]:
# create the feature daily_return based on close price
price_df['daily_return'] = price_df.groupby('ticker')['close'].transform(lambda x: x.pct_change())
price_df = price_df.dropna()

# Convert dates to datetime
news_df['publication_datetime'] = pd.to_datetime(news_df['publication_datetime'])
price_df['Date'] = pd.to_datetime(price_df['Date'])

In [22]:
# Initialize BERT tokenizer and model: https://huggingface.co/google-bert/bert-base-uncased
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()  # Set to evaluation mode

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [30]:
# Create a custom dataset
class NewsDataset(Dataset):
    def __init__(self, news_data, price_data, tokenizer, bert_model, max_length=512, device=None):
        self.news_data = news_data
        self.price_data = price_data
        self.tokenizer = tokenizer
        self.bert_model = bert_model
        self.max_length = max_length
        self.device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Create a mapping of (date, ticker) to daily return
        self.return_map = price_data.set_index(['Date', 'ticker'])['daily_return'].to_dict()
        
        # Pre-compute BERT embeddings for all articles
        self.embeddings = []
        
        print("Computing BERT embeddings for all articles...")
        with torch.no_grad():
            for idx in range(len(news_data)):
                if idx % 1000 == 0:
                    print(f"Processing article {idx}/{len(news_data)}")
                    
                text = news_data.iloc[idx]['body']
                inputs = self.tokenizer(text, 
                                      return_tensors='pt',
                                      max_length=self.max_length,
                                      padding='max_length',
                                      truncation=True)
                
                # 将输入数据移到与模型相同的设备上
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                
                outputs = self.bert_model(**inputs)
                # Use [CLS] token embedding as the document embedding
                embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
                self.embeddings.append(embedding)
                
        self.embeddings = np.array(self.embeddings)
        print("Finished computing embeddings.")
        
    def __len__(self):
        return len(self.news_data)
    
    def __getitem__(self, idx):
        news_row = self.news_data.iloc[idx]
        date = news_row['publication_datetime']
        ticker = news_row['tickers']
        
        # Get the next trading day's return
        next_trading_days = self.price_data[
            (self.price_data['Date'] >= date) & 
            (self.price_data['ticker'] == ticker)
        ]['Date'].unique()
        
        if len(next_trading_days) > 0:
            next_date = next_trading_days[0]
            daily_return = self.return_map.get((next_date, ticker), 0)
        else:
            daily_return = 0
            
        # Get pre-computed BERT embedding
        bert_embedding = self.embeddings[idx]
        
        return torch.FloatTensor(bert_embedding), torch.FloatTensor([daily_return])

In [31]:
# Define the neural network
class ReturnPredictor(nn.Module):
    def __init__(self, input_size=768, hidden_size=256):  # BERT base has 768 dimensions
        super(ReturnPredictor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size // 2, 1)
        )
    
    def forward(self, x):
        return self.network(x)

In [32]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Move BERT model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model = bert_model.to(device)

print("Creating datasets for rolling splits...")
# Create datasets based on the rolling splits defined earlier
datasets = {}

for i, split in enumerate(rolling_splits):
    print(f"Creating split {i+1}/{len(rolling_splits)}...")
    
    # Create train dataset for this split
    train_dataset_i = NewsDataset(
        split['train_set'],
        price_df,
        tokenizer,
        bert_model,
        device=device  # Pass device parameter
    )
    
    # Apply the same modification for validation and test datasets
    val_dataset_i = NewsDataset(
        split['val_set'],
        price_df,
        tokenizer,
        bert_model,
        device=device  # Pass device parameter
    )
    
    test_dataset_i = NewsDataset(
        split['test_set'],
        price_df,
        tokenizer,
        bert_model,
        device=device  # Pass device parameter
    )
    
    # Store datasets for this split
    datasets[f'split_{i+1}'] = {
        'train': train_dataset_i,
        'val': val_dataset_i,
        'test': test_dataset_i
    }

print(f"Created {len(datasets)} dataset splits")

Creating datasets for rolling splits...
Creating split 1/16...
Computing BERT embeddings for all articles...
Processing article 0/5015
Processing article 1000/5015
Processing article 2000/5015
Processing article 3000/5015
Processing article 4000/5015
Processing article 5000/5015
Finished computing embeddings.
Computing BERT embeddings for all articles...
Processing article 0/1839
Processing article 1000/1839
Finished computing embeddings.
Computing BERT embeddings for all articles...
Processing article 0/902
Finished computing embeddings.
Creating split 2/16...
Computing BERT embeddings for all articles...
Processing article 0/5103
Processing article 1000/5103
Processing article 2000/5103
Processing article 3000/5103
Processing article 4000/5103
Processing article 5000/5103
Finished computing embeddings.
Computing BERT embeddings for all articles...
Processing article 0/1916
Processing article 1000/1916
Finished computing embeddings.
Computing BERT embeddings for all articles...
Proces

In [33]:
# Create data loaders for each split
data_loaders = {}

for split_name, split_datasets in datasets.items():
    train_loader = DataLoader(split_datasets['train'], batch_size=32, shuffle=True)
    val_loader = DataLoader(split_datasets['val'], batch_size=32)
    test_loader = DataLoader(split_datasets['test'], batch_size=32)
    
    data_loaders[split_name] = {
        'train': train_loader,
        'val': val_loader,
        'test': test_loader
    }

print(f"Created data loaders for {len(data_loaders)} splits")

Created data loaders for 16 splits


In [36]:
# Train on each split
results = {}

for split_name, loaders in data_loaders.items():
    print(f"\nTraining on {split_name}")
    
    # Initialize model, criterion, optimizer for this split
    model = ReturnPredictor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    train_losses = []
    val_losses = []
    
    # Early stopping parameters
    min_epochs = 4  # Train for at least 4 epochs
    max_epochs = 10  # Train for at most 10 epochs
    patience = 3  # Stop if no improvement for 3 consecutive epochs
    min_delta = 0.001  # Minimum improvement threshold
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None
    
    train_loader = loaders['train']
    val_loader = loaders['val']
    test_loader = loaders['test']
    
    # Train the model
    total_training_time = 0
    for epoch in range(max_epochs):
        model.train()
        train_loss = 0
        epoch_start_time = time.time()
        
        # Add progress bar to display training progress
        train_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{max_epochs} [Train]')
        for batch_bert, batch_returns in train_progress:
            batch_bert = batch_bert.to(device)
            batch_returns = batch_returns.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_bert)
            loss = criterion(outputs, batch_returns)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            # Update progress bar with current loss
            train_progress.set_postfix(loss=f'{loss.item():.4f}')
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            # Add progress bar to display validation progress
            val_progress = tqdm(val_loader, desc=f'Epoch {epoch+1}/{max_epochs} [Val]')
            for batch_bert, batch_returns in val_progress:
                batch_bert = batch_bert.to(device)
                batch_returns = batch_returns.to(device)
                outputs = model(batch_bert)            
                current_loss = criterion(outputs, batch_returns).item()
                val_loss += current_loss
                # Update progress bar with current loss
                val_progress.set_postfix(loss=f'{current_loss:.4f}')
        
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - epoch_start_time
        total_training_time += epoch_duration
        
        train_loss = train_loss / len(train_loader)
        val_loss = val_loss / len(val_loader)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print(f'Epoch [{epoch+1}/{max_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {epoch_duration:.2f}s')
        
        # Early stopping logic
        if val_loss < best_val_loss - min_delta:
            # Significant improvement
            best_val_loss = val_loss
            epochs_no_improve = 0
            # Save best model state
            best_model_state = model.state_dict().copy()
        else:
            # No significant improvement
            epochs_no_improve += 1
            
        # Check if training should stop
        if epoch + 1 >= min_epochs and epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs. No improvement for {epochs_no_improve} epochs.")
            # Restore to best model state
            if best_model_state is not None:
                model.load_state_dict(best_model_state)
            break
    
    avg_epoch_time = total_training_time / (epoch + 1)
    print(f'Average epoch training time: {avg_epoch_time:.2f}s')
    
    # Test evaluation
    model.eval()
    test_loss = 0
    predictions = []
    actuals = []
    
    with torch.no_grad():
        # Add progress bar to display testing progress
        test_progress = tqdm(test_loader, desc=f'Testing {split_name}')
        for batch_bert, batch_returns in test_progress:
            batch_bert = batch_bert.to(device)
            batch_returns = batch_returns.to(device)
            outputs = model(batch_bert)
            current_loss = criterion(outputs, batch_returns).item()
            test_loss += current_loss
            
            predictions.extend(outputs.cpu().numpy().flatten())
            actuals.extend(batch_returns.cpu().numpy().flatten())
            # Update progress bar with current loss
            test_progress.set_postfix(loss=f'{current_loss:.4f}')
    
    test_loss = test_loss / len(test_loader)
    
    # Store results for this split
    results[split_name] = {
        'model': model,
        'train_losses': train_losses,
        'val_losses': val_losses,
        'test_loss': test_loss,
        'predictions': predictions,
        'actuals': actuals,
        'avg_epoch_time': avg_epoch_time, 
        'epochs_trained': epoch + 1
    }
    
    print(f"{split_name} Test Loss: {test_loss:.4f}, Epochs Trained: {epoch+1}")

print("\nTraining completed for all splits.")


Training on split_1


Epoch 1/10 [Val]: 100%|██████████| 58/58 [00:31<00:00,  1.83it/s, loss=0.0009]


Epoch [1/10], Train Loss: 0.0017, Val Loss: 0.0007, Time: 118.27s


Epoch 2/10 [Val]: 100%|██████████| 58/58 [00:31<00:00,  1.82it/s, loss=0.0010]


Epoch [2/10], Train Loss: 0.0009, Val Loss: 0.0007, Time: 118.97s


...

Epoch [3/10], Train Loss: 0.0019, Val Loss: 0.0011, Time: 116.22s


Epoch 4/10 [Val]: 100%|██████████| 54/54 [00:29<00:00,  1.83it/s, loss=0.0012]


Epoch [4/10], Train Loss: 0.0018, Val Loss: 0.0011, Time: 116.04s
Early stopping triggered after 4 epochs. No improvement for 3 epochs.
Average epoch training time: 116.05s


Testing split_16: 100%|██████████| 25/25 [00:13<00:00,  1.84it/s, loss=0.0003]

split_16 Test Loss: 0.0009, Epochs Trained: 4

Training completed for all splits.


In [37]:
def calculate_r2(y_true, y_pred, in_sample=True, benchmark=None):
    if in_sample:
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - np.mean(y_true)) ** 2))
    else:
        if benchmark is None:
            raise ValueError("Benchmark must be provided for out-of-sample R-squared calculation.")
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - benchmark) ** 2))

In [38]:
# Evaluate model performance across all splits
evaluation_results = {}

for split_name, split_data in results.items():
    model = split_data['model']
    test_loader = data_loaders[split_name]['test']
    
    # Evaluate model performance on the test set
    model.eval()
    test_predictions = []
    test_actuals = []
    
    with torch.no_grad():
        for batch_bert, batch_returns in test_loader:
            batch_bert = batch_bert.to(device)
            outputs = model(batch_bert)
            test_predictions.extend(outputs.cpu().numpy())
            test_actuals.extend(batch_returns.numpy())
    
    test_predictions = np.array(test_predictions)
    test_actuals = np.array(test_actuals)
    
    # Calculate R² and MSE
    r2 = calculate_r2(test_actuals, test_predictions, in_sample=False, benchmark=0)
    mse = np.mean((test_predictions - test_actuals) ** 2)
    
    # Store evaluation results
    evaluation_results[split_name] = {
        'r2': r2,
        'mse': mse
    }
    
    # Print evaluation metrics
    print(f"\n{split_name} Test Results:")
    print(f"R² Score: {r2:.6f}")
    print(f"MSE: {mse:.6f}")

# Create summary table of evaluation results
import pandas as pd

# Extract metrics for each split
summary_data = []
for split_name, metrics in evaluation_results.items():
    summary_data.append({
        'Split': split_name,
        'R²': metrics['r2'],
        'MSE': metrics['mse']
    })

# Create summary DataFrame
summary_df = pd.DataFrame(summary_data)
print("\nEvaluation Metrics Summary for All Splits:")
print(summary_df)


split_1 Test Results:
R² Score: -0.191368
MSE: 0.000427

split_2 Test Results:
R² Score: -0.130437
MSE: 0.000778

split_3 Test Results:
R² Score: -0.006831
MSE: 0.000876

split_4 Test Results:
R² Score: -0.043816
MSE: 0.001311

split_5 Test Results:
R² Score: -0.028990
MSE: 0.001772

split_6 Test Results:
R² Score: -0.235239
MSE: 0.000672

split_7 Test Results:
R² Score: -0.077370
MSE: 0.000508

split_8 Test Results:
R² Score: -0.125900
MSE: 0.000655

split_9 Test Results:
R² Score: -0.169601
MSE: 0.001461

split_10 Test Results:
R² Score: -0.096404
MSE: 0.000681

split_11 Test Results:
R² Score: -0.049841
MSE: 0.000709

split_12 Test Results:
R² Score: -0.010872
MSE: 0.004941

split_13 Test Results:
R² Score: -0.051972
MSE: 0.001390

split_14 Test Results:
R² Score: -0.026567
MSE: 0.001188

split_15 Test Results:
R² Score: -0.106183
MSE: 0.000793

split_16 Test Results:
R² Score: -0.318277
MSE: 0.000957

Evaluation Metrics Summary for All Splits:
       Split        R²       MSE
0   

In [39]:
# Save test set predictions for each split
all_test_dfs = []

for split_name, split_data in results.items():
    # Get corresponding test set data and predictions
    test_set = rolling_splits[int(split_name.split('_')[1])-1]['test_set']
    test_predictions = split_data['predictions']
    
    # Copy test data and add predictions as sentiment
    test_df = test_set.copy()
    test_df['sentiment'] = test_predictions
    
    # Keep only necessary columns
    keep_columns = ['publication_datetime', 'title', 'body', 'tickers', 'sentiment']
    test_df = test_df[keep_columns]
    
    # Add to the list
    all_test_dfs.append(test_df)

# Merge all test data
if all_test_dfs:
    all_test_predictions = pd.concat(all_test_dfs, ignore_index=True)
    
    # Sort by date
    all_test_predictions['publication_datetime'] = pd.to_datetime(all_test_predictions['publication_datetime'])
    all_test_predictions = all_test_predictions.sort_values('publication_datetime')
    
    # Save the merged predictions
    all_test_predictions.to_csv("news_w_sentiment_FineTuneBERT.csv", index=False)
    print(f"All predictions saved to news_w_sentiment_FineTuneBERT.csv, total records: {len(all_test_predictions)}")
    
    # Display the first few rows to preview the results
    print("\nData preview:")
    print(all_test_predictions.head())

All predictions saved to news_w_sentiment_FineTuneBERT.csv, total records: 13696

Data preview:
   publication_datetime                                              title  \
0            2018-05-01                                Still Hot For Cocoa   
22           2018-05-01                      T-Mobile Must Stay a Maverick   
21           2018-05-01  Business News: Deal to Forge Biggest U.S. Refiner   
20           2018-05-01  U.S. News: High Court to Heighten Class-Action...   
19           2018-05-01             Walmart Pulls Back From U.K. Groceries   

                                                 body tickers  sentiment  
0   That compares with aroughly 7% gain for a Gold...     HSY   0.000272  
22  The politically important sideshow will be whe...    TMUS   0.012052  
21  Marathon's shares have nearly tripled since th...     MPC   0.002517  
20  But rather than try to distribute that money a...   GOOGL  -0.001273  
19  Walmart bought the U.K.'s third-largest grocer...     WM

In [40]:
import os
import pickle
import numpy as np
import json
import matplotlib.pyplot as plt

# Create directory to save results
os.makedirs("fine-tuned_bert_model_results", exist_ok=True)

# Save results for each split
for split_name, split_result in results.items():
    # Create subdirectory for each split
    split_dir = os.path.join("fine-tuned_bert_model_results", split_name)
    os.makedirs(split_dir, exist_ok=True)
    
    # 1. Save model weights
    model_path = os.path.join(split_dir, f"{split_name}_model.pth")
    torch.save(split_result['model'].state_dict(), model_path)
    
    # 2. Save training and validation loss history
    losses = {
        'train_losses': split_result['train_losses'],
        'val_losses': split_result['val_losses'],
        'test_loss': split_result['test_loss']
    }
    
    # Convert NumPy arrays to Python lists for JSON serialization
    for key in losses:
        if isinstance(losses[key], (np.ndarray, list)):
            losses[key] = [float(x) for x in losses[key]]
        elif isinstance(losses[key], (np.float32, np.float64)):
            losses[key] = float(losses[key])
    
    # Save loss data
    with open(os.path.join(split_dir, f"{split_name}_losses.json"), 'w') as f:
        json.dump(losses, f, indent=4)
    
    # 3. Save prediction results
    predictions_data = {
        'predictions': [float(p) for p in split_result['predictions']],
        'actuals': [float(a) for a in split_result['actuals']]
    }
    with open(os.path.join(split_dir, f"{split_name}_predictions.json"), 'w') as f:
        json.dump(predictions_data, f, indent=4)
    
    # 4. Plot and save loss curves
    plt.figure(figsize=(10, 6))
    plt.plot(split_result['train_losses'], label='Train Loss')
    plt.plot(split_result['val_losses'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'{split_name} - Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(split_dir, f"{split_name}_loss_curve.png"))
    plt.close()
    
    # 5. Plot prediction vs actual values scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(split_result['actuals'], split_result['predictions'], alpha=0.5)
    plt.plot([-0.1, 0.1], [-0.1, 0.1], 'r--')  # Ideal line
    plt.xlabel('Actual Returns')
    plt.ylabel('Predicted Returns')
    plt.title(f'{split_name} - Prediction vs Actual')
    plt.grid(True)
    plt.savefig(os.path.join(split_dir, f"{split_name}_prediction_scatter.png"))
    plt.close()

# Create README file with explanation of results
readme_content = """# Financial News Sentiment Analysis Model Results

## Model Structure
- BERT model for extracting news text features
- Fully connected layers for stock return prediction
- Trained using MSE loss function and Adam optimizer

## File Descriptions
Each time window split contains the following files:
- `{split_name}_model.pth`: Trained model weights
- `{split_name}_losses.json`: Training and validation loss values
- `{split_name}_predictions.json`: Predictions and actual values on test set
- `{split_name}_loss_curve.png`: Loss curve chart
- `{split_name}_prediction_scatter.png`: Prediction vs actual values scatter plot

## How to Load and Use the Model"""

with open(os.path.join("fine-tuned_bert_model_results", "README.md"), 'w') as f:
    f.write(readme_content)

print("All model results saved to 'fine-tuned_bert_model_results' directory")


All model results saved to 'fine-tuned_bert_model_results' directory


# Fine-tune FinBERT

In [56]:
# Import additional required libraries
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
import torch.nn as nn
import time
from tqdm import tqdm 

# Load FinBERT model and tokenizer
print("Loading FinBERT model and tokenizer...")
finbert_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')
finbert_model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain')
finbert_model.eval()  # Set to evaluation mode


Loading FinBERT model and tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [65]:
import torch
from torch.utils.data import Dataset
import numpy as np

class FinBERTNewsDataset(Dataset):
    def __init__(self, news_data, price_data, tokenizer, finbert_model, max_length=512, device=None):
        self.news_data = news_data
        self.price_data = price_data
        self.tokenizer = tokenizer
        self.finbert_model = finbert_model
        self.max_length = max_length
        self.device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Create a mapping of (date, ticker) to daily return
        self.return_map = price_data.set_index(['Date', 'ticker'])['daily_return'].to_dict()
        
        # Pre-compute FinBERT embeddings for all articles
        self.embeddings = []
        
        print("Computing FinBERT embeddings for all articles...")
        with torch.no_grad():
            for idx in range(len(news_data)):
                if idx % 1000 == 0:
                    print(f"Processing article {idx}/{len(news_data)}")
                    
                text = news_data.iloc[idx]['body']
                inputs = self.tokenizer(text, 
                                        return_tensors='pt',
                                        max_length=self.max_length,
                                        padding='max_length',
                                        truncation=True)
                
                # Move inputs to the same device as the model
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                
                outputs = self.finbert_model(**inputs, output_hidden_states=True)
                # Use [CLS] token embedding from the last hidden state as document embedding
                embedding = outputs.hidden_states[-1][:, 0, :].squeeze().cpu().numpy()
                self.embeddings.append(embedding)
                
        self.embeddings = np.array(self.embeddings)
        print("Finished computing embeddings.")
        
    def __len__(self):
        return len(self.news_data)
    
    def __getitem__(self, idx):
        news_row = self.news_data.iloc[idx]
        date = news_row['publication_datetime']
        ticker = news_row['tickers']
        
        # Get the next trading day's return
        next_trading_days = self.price_data[
            (self.price_data['Date'] >= date) & 
            (self.price_data['ticker'] == ticker)
        ]['Date'].unique()
        
        if len(next_trading_days) > 0:
            next_date = next_trading_days[0]
            daily_return = self.return_map.get((next_date, ticker), 0)
        else:
            daily_return = 0
            
        # Get pre-computed FinBERT embedding
        finbert_embedding = self.embeddings[idx]
        
        return torch.FloatTensor(finbert_embedding), torch.FloatTensor([daily_return])

In [66]:
# Define the neural network for FinBERT
class FinBERTReturnPredictor(nn.Module):
    def __init__(self, input_size=768, hidden_size=256):  # FinBERT base also has 768 dimensions
        super(FinBERTReturnPredictor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size // 2, 1)
        )
    
    def forward(self, x):
        return self.network(x)

In [69]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Move FinBERT model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
finbert_model.to(device)

print("Creating datasets for rolling splits...")
# Create datasets based on the rolling splits defined earlier
finbert_datasets = {}

for i, split in enumerate(rolling_splits):
    print(f"Creating split {i+1}/{len(rolling_splits)}...")
    
    # Create train dataset for this split
    train_dataset_i = FinBERTNewsDataset(
        split['train_set'],
        price_df,
        finbert_tokenizer,
        finbert_model,
        device=device  # 传入设备参数
    )
    
    # Create validation dataset for this split
    val_dataset_i = FinBERTNewsDataset(
        split['val_set'],
        price_df,
        finbert_tokenizer,
        finbert_model,
        device=device  # 传入设备参数
    )
    
    # Create test dataset for this split
    test_dataset_i = FinBERTNewsDataset(
        split['test_set'],
        price_df,
        finbert_tokenizer,
        finbert_model,
        device=device  # 传入设备参数
    )
    
    # Store datasets for this split
    finbert_datasets[f'split_{i+1}'] = {
        'train': train_dataset_i,
        'val': val_dataset_i,
        'test': test_dataset_i
    }

print(f"Created {len(finbert_datasets)} dataset splits")

Creating datasets for rolling splits...
Creating split 1/16...
Computing FinBERT embeddings for all articles...
Processing article 0/5015
Processing article 1000/5015
Processing article 2000/5015
Processing article 3000/5015
Processing article 4000/5015
Processing article 5000/5015
Finished computing embeddings.
Computing FinBERT embeddings for all articles...
Processing article 0/1839
Processing article 1000/1839
Finished computing embeddings.
Computing FinBERT embeddings for all articles...
Processing article 0/902
Finished computing embeddings.
Creating split 2/16...
Computing FinBERT embeddings for all articles...
Processing article 0/5103
Processing article 1000/5103
Processing article 2000/5103
Processing article 3000/5103
Processing article 4000/5103
Processing article 5000/5103
Finished computing embeddings.
Computing FinBERT embeddings for all articles...
Processing article 0/1916
Processing article 1000/1916
Finished computing embeddings.
Computing FinBERT embeddings for all 

In [70]:
# Create data loaders for each split
finbert_data_loaders = {}

for split_name, split_datasets in finbert_datasets.items():
    train_loader = DataLoader(split_datasets['train'], batch_size=32, shuffle=True)
    val_loader = DataLoader(split_datasets['val'], batch_size=32)
    test_loader = DataLoader(split_datasets['test'], batch_size=32)
    
    finbert_data_loaders[split_name] = {
        'train': train_loader,
        'val': val_loader,
        'test': test_loader
    }

print(f"Created data loaders for {len(finbert_data_loaders)} splits")

Created data loaders for 16 splits


In [71]:
# Train on each split
finbert_results = {}

for split_name, loaders in finbert_data_loaders.items():
    print(f"\nTraining on {split_name}")
    
    # Initialize model, criterion, optimizer for this split
    model = FinBERTReturnPredictor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    train_losses = []
    val_losses = []
    
    # Early stopping parameters
    min_epochs = 4  # Train for at least 4 epochs
    max_epochs = 10  # Train for at most 10 epochs
    patience = 3  # Stop if no improvement for 3 consecutive epochs
    min_delta = 0.001  # Minimum improvement threshold
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None
    
    train_loader = loaders['train']
    val_loader = loaders['val']
    test_loader = loaders['test']
    
    # Train the model
    total_training_time = 0
    for epoch in range(max_epochs):
        model.train()
        train_loss = 0
        epoch_start_time = time.time()
        
        # Add progress bar to display training progress
        train_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{max_epochs} [Train]')
        for batch_finbert, batch_returns in train_progress:
            batch_finbert = batch_finbert.to(device)
            batch_returns = batch_returns.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_finbert)
            loss = criterion(outputs, batch_returns)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            # Update progress bar with current loss
            train_progress.set_postfix(loss=f'{loss.item():.4f}')
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            # Add progress bar to display validation progress
            val_progress = tqdm(val_loader, desc=f'Epoch {epoch+1}/{max_epochs} [Val]')
            for batch_finbert, batch_returns in val_progress:
                batch_finbert = batch_finbert.to(device)
                batch_returns = batch_returns.to(device)
                outputs = model(batch_finbert)            
                current_loss = criterion(outputs, batch_returns).item()
                val_loss += current_loss
                # Update progress bar with current loss
                val_progress.set_postfix(loss=f'{current_loss:.4f}')
        
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - epoch_start_time
        total_training_time += epoch_duration
        
        train_loss = train_loss / len(train_loader)
        val_loss = val_loss / len(val_loader)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print(f'Epoch [{epoch+1}/{max_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {epoch_duration:.2f}s')
        
        # Early stopping logic
        if val_loss < best_val_loss - min_delta:
            # Significant improvement
            best_val_loss = val_loss
            epochs_no_improve = 0
            # Save best model state
            best_model_state = model.state_dict().copy()
        else:
            # No significant improvement
            epochs_no_improve += 1
            
        # Check if training should stop
        if epoch + 1 >= min_epochs and epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs. No improvement for {epochs_no_improve} epochs.")
            # Restore to best model state
            if best_model_state is not None:
                model.load_state_dict(best_model_state)
            break
    
    avg_epoch_time = total_training_time / (epoch + 1)
    print(f'Average epoch training time: {avg_epoch_time:.2f}s')
    
    # Test evaluation
    model.eval()
    test_loss = 0
    predictions = []
    actuals = []
    
    with torch.no_grad():
        # Add progress bar to display testing progress
        test_progress = tqdm(test_loader, desc=f'Testing {split_name}')
        for batch_finbert, batch_returns in test_progress:
            batch_finbert = batch_finbert.to(device)
            batch_returns = batch_returns.to(device)
            outputs = model(batch_finbert)
            current_loss = criterion(outputs, batch_returns).item()
            test_loss += current_loss
            
            predictions.extend(outputs.cpu().numpy().flatten())
            actuals.extend(batch_returns.cpu().numpy().flatten())
            # Update progress bar with current loss
            test_progress.set_postfix(loss=f'{current_loss:.4f}')
    
    test_loss = test_loss / len(test_loader)
    
    # Store results for this split
    finbert_results[split_name] = {
        'model': model,
        'train_losses': train_losses,
        'val_losses': val_losses,
        'test_loss': test_loss,
        'predictions': predictions,
        'actuals': actuals,
        'avg_epoch_time': avg_epoch_time,
        'epochs_trained': epoch + 1
    }
    
    print(f"{split_name} Test Loss: {test_loss:.4f}, Epochs Trained: {epoch+1}")

print("\nTraining completed for all splits.")


Training on split_1


Epoch 1/10 [Val]: 100%|██████████| 58/58 [00:31<00:00,  1.86it/s, loss=0.0011]


Epoch [1/10], Train Loss: 0.0045, Val Loss: 0.0008, Time: 116.28s


Epoch 2/10 [Val]: 100%|██████████| 58/58 [00:31<00:00,  1.86it/s, loss=0.0012]


Epoch [2/10], Train Loss: 0.0016, Val Loss: 0.0007, Time: 116.88s


...

Epoch [3/10], Train Loss: 0.0020, Val Loss: 0.0010, Time: 115.43s


Epoch 4/10 [Val]: 100%|██████████| 54/54 [00:29<00:00,  1.85it/s, loss=0.0006]


Epoch [4/10], Train Loss: 0.0018, Val Loss: 0.0010, Time: 114.92s
Early stopping triggered after 4 epochs. No improvement for 3 epochs.
Average epoch training time: 115.48s


Testing split_16: 100%|██████████| 25/25 [00:13<00:00,  1.86it/s, loss=0.0001]

split_16 Test Loss: 0.0007, Epochs Trained: 4

Training completed for all splits.


In [72]:
def calculate_r2(y_true, y_pred, in_sample=True, benchmark=None):
    if in_sample:
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - np.mean(y_true)) ** 2))
    else:
        if benchmark is None:
            raise ValueError("Benchmark must be provided for out-of-sample R-squared calculation.")
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - benchmark) ** 2))

In [73]:
# Evaluate model performance across all splits
finbert_evaluation_results = {}

for split_name, split_data in finbert_results.items():
    model = split_data['model']
    test_loader = finbert_data_loaders[split_name]['test']
    
    # Evaluate model performance on the test set
    model.eval()
    test_predictions = []
    test_actuals = []
    
    with torch.no_grad():
        for batch_finbert, batch_returns in test_loader:
            batch_finbert = batch_finbert.to(device)
            outputs = model(batch_finbert)
            test_predictions.extend(outputs.cpu().numpy())
            test_actuals.extend(batch_returns.numpy())
    
    test_predictions = np.array(test_predictions)
    test_actuals = np.array(test_actuals)
    
    # Calculate R² and MSE
    r2 = calculate_r2(test_actuals, test_predictions, in_sample=False, benchmark=0)
    mse = np.mean((test_predictions - test_actuals) ** 2)
    
    # Store evaluation results
    finbert_evaluation_results[split_name] = {
        'r2': r2,
        'mse': mse
    }
    
    # Print evaluation metrics
    print(f"\n{split_name} Test Results:")
    print(f"R² Score: {r2:.6f}")
    print(f"MSE: {mse:.6f}")

# Create summary table of evaluation results
import pandas as pd

# Extract metrics for each split
summary_data = []
for split_name, metrics in finbert_evaluation_results.items():
    summary_data.append({
        'Split': split_name,
        'R²': metrics['r2'],
        'MSE': metrics['mse']
    })

# Create summary DataFrame
summary_df = pd.DataFrame(summary_data)
print("\nEvaluation Metrics Summary for All Splits:")
print(summary_df)


split_1 Test Results:
R² Score: -0.212674
MSE: 0.000435

split_2 Test Results:
R² Score: -0.015538
MSE: 0.000699

split_3 Test Results:
R² Score: -0.174191
MSE: 0.001021

split_4 Test Results:
R² Score: -0.287692
MSE: 0.001618

split_5 Test Results:
R² Score: -0.074915
MSE: 0.001851

split_6 Test Results:
R² Score: -0.096674
MSE: 0.000597

split_7 Test Results:
R² Score: -0.295921
MSE: 0.000612

split_8 Test Results:
R² Score: -0.208950
MSE: 0.000703

split_9 Test Results:
R² Score: -0.080251
MSE: 0.001350

split_10 Test Results:
R² Score: -0.086370
MSE: 0.000675

split_11 Test Results:
R² Score: -0.124485
MSE: 0.000759

split_12 Test Results:
R² Score: -0.053566
MSE: 0.005150

split_13 Test Results:
R² Score: -0.060558
MSE: 0.001401

split_14 Test Results:
R² Score: -0.127904
MSE: 0.001305

split_15 Test Results:
R² Score: -0.079212
MSE: 0.000774

split_16 Test Results:
R² Score: -0.008780
MSE: 0.000732

Evaluation Metrics Summary for All Splits:
       Split        R²       MSE
0   

In [74]:
# Save test set predictions for each split
all_test_dfs = []

for split_name, split_data in finbert_results.items():
    # Get corresponding test set data and predictions
    test_set = rolling_splits[int(split_name.split('_')[1])-1]['test_set']
    test_predictions = split_data['predictions']
    
    # Copy test data and add predictions as sentiment
    test_df = test_set.copy()
    test_df['sentiment'] = test_predictions
    
    # Keep only necessary columns
    keep_columns = ['publication_datetime', 'title', 'body', 'tickers', 'sentiment']
    test_df = test_df[keep_columns]
    
    # Add to the list
    all_test_dfs.append(test_df)

# Merge all test data
if all_test_dfs:
    all_test_predictions = pd.concat(all_test_dfs, ignore_index=True)
    
    # Sort by date
    all_test_predictions['publication_datetime'] = pd.to_datetime(all_test_predictions['publication_datetime'])
    all_test_predictions = all_test_predictions.sort_values('publication_datetime')
    
    # Save the merged predictions
    all_test_predictions.to_csv("news_w_sentiment_FineTuneFinBERT.csv", index=False)
    print(f"All predictions saved to news_w_sentiment_FineTuneFinBERT.csv, total records: {len(all_test_predictions)}")
    
    # Display the first few rows to preview the results
    print("\nData preview:")
    print(all_test_predictions.head())

All predictions saved to news_w_sentiment_FineTuneFinBERT.csv, total records: 13696

Data preview:
   publication_datetime                                              title  \
0            2018-05-01                                Still Hot For Cocoa   
22           2018-05-01                      T-Mobile Must Stay a Maverick   
21           2018-05-01  Business News: Deal to Forge Biggest U.S. Refiner   
20           2018-05-01  U.S. News: High Court to Heighten Class-Action...   
19           2018-05-01             Walmart Pulls Back From U.K. Groceries   

                                                 body tickers  sentiment  
0   That compares with aroughly 7% gain for a Gold...     HSY  -0.002577  
22  The politically important sideshow will be whe...    TMUS   0.002361  
21  Marathon's shares have nearly tripled since th...     MPC   0.007085  
20  But rather than try to distribute that money a...   GOOGL  -0.005097  
19  Walmart bought the U.K.'s third-largest grocer...    

In [75]:
import os
import pickle
import numpy as np
import json
import matplotlib.pyplot as plt

# Create directory to save results
os.makedirs("fine-tuned_finbert_model_results", exist_ok=True)

# Save results for each split
for split_name, split_result in finbert_results.items():
    # Create subdirectory for each split
    split_dir = os.path.join("fine-tuned_finbert_model_results", split_name)
    os.makedirs(split_dir, exist_ok=True)
    
    # 1. Save model weights
    model_path = os.path.join(split_dir, f"{split_name}_model.pth")
    torch.save(split_result['model'].state_dict(), model_path)
    
    # 2. Save training and validation loss history
    losses = {
        'train_losses': split_result['train_losses'],
        'val_losses': split_result['val_losses'],
        'test_loss': split_result['test_loss']
    }
    
    # Convert NumPy arrays to Python lists for JSON serialization
    for key in losses:
        if isinstance(losses[key], (np.ndarray, list)):
            losses[key] = [float(x) for x in losses[key]]
        elif isinstance(losses[key], (np.float32, np.float64)):
            losses[key] = float(losses[key])
    
    # Save loss data
    with open(os.path.join(split_dir, f"{split_name}_losses.json"), 'w') as f:
        json.dump(losses, f, indent=4)
    
    # 3. Save prediction results
    predictions_data = {
        'predictions': [float(p) for p in split_result['predictions']],
        'actuals': [float(a) for a in split_result['actuals']]
    }
    with open(os.path.join(split_dir, f"{split_name}_predictions.json"), 'w') as f:
        json.dump(predictions_data, f, indent=4)
    
    # 4. Plot and save loss curves
    plt.figure(figsize=(10, 6))
    plt.plot(split_result['train_losses'], label='Train Loss')
    plt.plot(split_result['val_losses'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'{split_name} - Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(split_dir, f"{split_name}_loss_curve.png"))
    plt.close()
    
    # 5. Plot prediction vs actual values scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(split_result['actuals'], split_result['predictions'], alpha=0.5)
    plt.plot([-0.1, 0.1], [-0.1, 0.1], 'r--')  # Ideal line
    plt.xlabel('Actual Returns')
    plt.ylabel('Predicted Returns')
    plt.title(f'{split_name} - Prediction vs Actual')
    plt.grid(True)
    plt.savefig(os.path.join(split_dir, f"{split_name}_prediction_scatter.png"))
    plt.close()

# Create README file with explanation of results
readme_content = """# Financial News Sentiment Analysis with FinBERT

## Model Structure
- FinBERT model for extracting news text features
- Fully connected layers for stock return prediction
- Trained using MSE loss function and Adam optimizer

## File Descriptions
Each time window split contains the following files:
- `{split_name}_model.pth`: Trained model weights
- `{split_name}_losses.json`: Training and validation loss values
- `{split_name}_predictions.json`: Predictions and actual values on test set
- `{split_name}_loss_curve.png`: Loss curve chart
- `{split_name}_prediction_scatter.png`: Prediction vs actual values scatter plot

## How to Load and Use the Model
"""

with open(os.path.join("fine-tuned_finbert_model_results", "README.md"), 'w') as f:
    f.write(readme_content)

print("All model results saved to 'fine-tuned_finbert_model_results' directory")

All model results saved to 'fine-tuned_finbert_model_results' directory
